In [ ]:
import h5py 
import dxchange
import os
import numpy as np
from holotomocupy.utils import *

## Parameters

In [ ]:
n = 2048
ntheta = 4500
detector_pixelsize = 1.4760147601476e-6 * 2
energy = 17.1
wavelength = 1.24e-09 / energy
focustodetectordistance = 1.217

sx0 = -3.135e-3
z1 = np.array([5.110, 5.464, 6.879, 9.817, 10.372, 11.146, 12.594, 17.209]) * 1e-3 - sx0
z1_ids = np.array([0, 1, 2, 3]) ### note using index starting from 0
str_z1_ids = ''.join(map(str, z1_ids + 1)) 
z1 = z1[z1_ids]
ndist = len(z1)
z2 = focustodetectordistance - z1

distances = (z1 * z2) / focustodetectordistance
magnifications = focustodetectordistance / z1
norm_magnifications = magnifications / magnifications[0]
voxelsizes = np.abs(detector_pixelsize / magnifications)
voxelsize = voxelsizes[0]

path = '/data2/vnikitin/brain/20251115/'
pfile = 'Y350a_HT_20nm_8dist'
path_out = '/data2/vnikitin/brain_rec/20251115/Y350a'
file_out = f'data{str_z1_ids}.h5'


In [ ]:
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

import matplotlib as mpl

mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],  # fallbacks
})

mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "mathtext.fontset": "stix",   # Times-like math
    # optional:
    # "mathtext.default": "regular",
})
def mshow(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        cbar = fig.colorbar(im, fraction=0.046, pad=0.04)
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        cbar.ax.tick_params(labelsize=28)
        axs.add_artist(scalebar)
        # plt.show()


### Parse files and save everything to h5

In [ ]:
import numpy as np
import tifffile
from holotomocupy.utils import *

path = '/data2/vnikitin/brain/20251115/Y350a_HT_20nm_8dist_rec1234_/Y350a_HT_20nm_8dist_rec_cm_.vol'
nx, ny, nz = 3216, 3216, 2048
dtype = np.dtype("<f4")

k = 0  # z index

bytes_per_voxel = dtype.itemsize
slice_bytes = ny * nx *nz* bytes_per_voxel
offset = k * slice_bytes

with open(path, "rb") as f:
    f.seek(offset, 0)
    buf = f.read(slice_bytes)

prec = np.frombuffer(buf, dtype=dtype).reshape(nz,ny, nx)

In [ ]:
iter=64
mrec = tifffile.imread(f'/data2//vnikitin/brain_rec/20251115/Y350a1234/4500_2048_0_0.0_0.003_0.05_0.02_20_1.1_0/rec_obj_real/{iter:04}.tiff')
mrec = mrec[mrec.shape[0]//2-nz//2:mrec.shape[0]//2+nz//2,
            mrec.shape[1]//2-ny//2:mrec.shape[1]//2+ny//2,
            mrec.shape[2]//2-nx//2:mrec.shape[2]//2+nx//2]


In [ ]:
mpl.rcParams['font.size'] = 24  # optional
mpl.rcParams["xtick.labelsize"] = 16
mpl.rcParams["ytick.labelsize"] = 16
def mshow(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        cbar = fig.colorbar(im, fraction=0.046, pad=0.1,orientation="horizontal", location="bottom")
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        cbar.ax.tick_params(labelsize=20)
        # axs.axis('off')
        axs.add_artist(scalebar)
        
        # plt.show()
def find_min_max(data):
    """Find min and max values according to histogram"""

    h, e = np.histogram(data[:], 1000)
    stend = np.where(h > np.max(h)*0.03)
    st = stend[0][0]
    end = stend[0][-1]
    mmin = e[st]
    mmax = e[end+1]

    return mmin, mmax

In [ ]:
aa=prec[nz//2,ny//2-512:ny//2+512,ny//2-512:ny//2+512]
bb=mrec[nz//2,ny//2-512:ny//2+512,ny//2-512:ny//2+512]

mmin1,mmax1 = find_min_max(aa)
mmin2,mmax2 = find_min_max(bb)

aa=prec[nz//2]
bb=mrec[nz//2]

aa = (aa-mmin1)/(mmax1-mmin1)
bb = (bb-mmin2)/(mmax2-mmin2)
aa[aa<0] = 0
aa[aa>1] = 1
bb[bb<0] = 0
bb[bb>1] = 1
mshow(aa,True,vmax=1,vmin=0)
plt.savefig(f"figs/precz.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow(bb,True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecz.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


aa=prec[:,ny//2]
bb=mrec[:,ny//2]

aa = (aa-mmin1)/(mmax1-mmin1)
bb = (bb-mmin2)/(mmax2-mmin2)
aa[aa<0] = 0
aa[aa>1] = 1
bb[bb<0] = 0
bb[bb>1] = 1
mshow(aa,True,vmax=1,vmin=0)
plt.savefig(f"figs/precy.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow(bb,True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecy.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

In [ ]:
def mshow1(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        # cbar = fig.colorbar(im, fraction=0.046, pad=0.1)
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        # cbar.ax.tick_params(labelsize=26)
        axs.axis('off')
        axs.add_artist(scalebar)
        # plt.show()

In [ ]:
aa=prec[nz//2]
bb=mrec[nz//2]

aa = (aa-mmin1)/(mmax1-mmin1)
bb = (bb-mmin2)/(mmax2-mmin2)
aa[aa<0] = 0
aa[aa>1] = 1
bb[bb<0] = 0
bb[bb>1] = 1

s=320
st=1200
mshow1(aa[ny//2-s:ny//2+3*s,nx//2-s+st:nx//2+s+st],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/preczz.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(bb[ny//2-s:ny//2+3*s,nx//2-s+st:nx//2+s+st],True,vmax=1,vmin=0)
plt.savefig(f"figs/mreczz.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


s=160
mshow1(aa[ny//2-s:ny//2+s,nx//2-s:nx//2+s],True,vmax=1/1.3,vmin=0)
plt.savefig(f"figs/precz0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(bb[ny//2-s:ny//2+s,nx//2-s:nx//2+s],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecz0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

stx = 1000
sty = 300

mshow1(aa[ny//2-s+sty:ny//2+s+sty,nx//2-s+stx:nx//2+s+stx],True,vmax=1/1.3,vmin=0)
plt.savefig(f"figs/precz1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(bb[ny//2-s+sty:ny//2+s+sty,nx//2-s+stx:nx//2+s+stx],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecz1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


aa=prec[:,ny//2]
bb=mrec[:,ny//2]

aa = (aa-mmin1)/(mmax1-mmin1)
bb = (bb-mmin2)/(mmax2-mmin2)
aa[aa<0] = 0
aa[aa>1] = 1
bb[bb<0] = 0
bb[bb>1] = 1

s=320
st=1200
mshow1(aa[nz//2-2*s:nz//2+2*s,nx//2-s+st:nx//2+s+st],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/precyy.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(bb[nz//2-2*s:nz//2+2*s,nx//2-s+st:nx//2+s+st],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecyy.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

s=160
mshow1(aa[nz//2-s:nz//2+s,nx//2-s:nx//2+s],True,vmax=1/1.3,vmin=0)
plt.savefig(f"figs/precy0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(bb[nz//2-s:nz//2+s,nx//2-s:nx//2+s],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecy0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

stx = 1000
sty = 300

mshow1(aa[nz//2-s+sty:nz//2+s+sty,nx//2-s+stx:nx//2+s+stx],True,vmax=1/1.3,vmin=0)
plt.savefig(f"figs/precy1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(bb[nz//2-s+sty:nz//2+s+sty,nx//2-s+stx:nx//2+s+stx],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecy1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


In [ ]:
def mshow1(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        # cbar = fig.colorbar(im, fraction=0.046, pad=0.1)
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        # cbar.ax.tick_params(labelsize=26)
        axs.axis('off')
        # axs.add_artist(scalebar)
        # plt.show()

In [ ]:
ss = 128
middle = [nz//2,ny//2+64,nx//2-60]
aa = prec[middle[0]-ss:middle[0]+ss,middle[1]-ss:middle[1]+ss,middle[2]-ss:middle[2]+ss]
aa = (aa-mmin1)/(mmax1-mmin1)
aa[aa<0] = 0
aa[aa>1] = 1

bb = mrec[middle[0]-ss:middle[0]+ss,middle[1]-ss:middle[1]+ss,middle[2]-ss:middle[2]+ss]
bb = (bb-mmin2)/(mmax2-mmin2)
bb[bb<0] = 0
bb[bb>1] = 1

mshow1(aa[ss],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/preczp0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow1(bb[ss],True,vmax=1,vmin=0)
plt.savefig(f"figs/mreczp0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(aa[:,ss],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/precyp0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow1(bb[:,ss],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecyp0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(aa[:,:,ss],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/precxp0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow1(bb[:,:,ss],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecxp0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


In [ ]:
# ss

In [ ]:
ss = 128
middle = [nz//2,2500+60,2100+90]
aa = prec[middle[0]-ss:middle[0]+ss,middle[1]-ss:middle[1]+ss,middle[2]-ss:middle[2]+ss]
aa = (aa-mmin1)/(mmax1-mmin1)
aa[aa<0] = 0
aa[aa>1] = 1

bb = mrec[middle[0]-ss:middle[0]+ss,middle[1]-ss:middle[1]+ss,middle[2]-ss:middle[2]+ss]
bb = (bb-mmin2)/(mmax2-mmin2)
bb[bb<0] = 0
bb[bb>1] = 1

mshow1(aa[ss],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/preczp1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow1(bb[ss],True,vmax=1,vmin=0)
plt.savefig(f"figs/mreczp1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(aa[:,ss],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/precyp1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow1(bb[:,ss],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecyp1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

mshow1(aa[:,:,ss],True,vmax=0.8,vmin=0)
plt.savefig(f"figs/precxp1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow1(bb[:,:,ss],True,vmax=1,vmin=0)
plt.savefig(f"figs/mrecxp1.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


In [ ]:

def mshow1(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        cbar = fig.colorbar(im, fraction=0.046, pad=0.1)
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        cbar.ax.tick_params(labelsize=24)
        axs.axis('off')
        axs.add_artist(scalebar)
        # plt.show()

In [ ]:
mpl.rcParams['font.size'] = 24  # optional
mpl.rcParams["xtick.labelsize"] = 16
mpl.rcParams["ytick.labelsize"] = 16
def mshow(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        cbar = fig.colorbar(im, fraction=0.046, pad=0.05)
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        cbar.ax.tick_params(labelsize=20)
        # axs.axis('off')
        axs.add_artist(scalebar)
        
        # plt.show()

def mshow1(a, show=False, **args):
    if show:
        if isinstance(a, cp.ndarray):
            a = a.get()
        fig, axs = plt.subplots(1, 1, figsize=(6,6))
        im = axs.imshow(a, cmap="gray", **args)
        cbar = fig.colorbar(im, fraction=0.046, pad=0.1)
        scalebar = ScaleBar(20e-3, "um", length_fraction=0.25, font_properties={
            "family": "serif",
        },  # For more information, see the cell below                
        location="lower right")
        cbar.ax.tick_params(labelsize=26)
        axs.axis('off')
        axs.add_artist(scalebar)
        # plt.show()

In [ ]:
mpl.rcParams['font.size'] = 24  # optional
mpl.rcParams["xtick.labelsize"] = 20
mpl.rcParams["ytick.labelsize"] = 20
iter=64+16
prb_abs = tifffile.imread(f'/data2//vnikitin/brain_rec/20251115/Y350a1234/4500_2048_0_0.0_0.003_0.05_0.02_20_1.1_0/rec_prb_abs0/{iter:04}.tiff')
prb_angle = tifffile.imread(f'/data2//vnikitin/brain_rec/20251115/Y350a1234/4500_2048_0_0.0_0.003_0.05_0.02_20_1.1_0/rec_prb_angle0/{iter:04}.tiff')
mshow(prb_abs*5000,True,vmax=7000,vmin=1000)
plt.savefig(f"figs/prbabs0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow(prb_angle,True,vmax=2.4,vmin=-2.4)
plt.savefig(f"figs/prbangle0.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


prb_abs = tifffile.imread(f'/data2//vnikitin/brain_rec/20251115/Y350a1234/4500_2048_0_0.0_0.003_0.05_0.02_20_1.1_0/rec_prb_abs3/{iter:04}.tiff')
prb_angle = tifffile.imread(f'/data2//vnikitin/brain_rec/20251115/Y350a1234/4500_2048_0_0.0_0.003_0.05_0.02_20_1.1_0/rec_prb_angle3/{iter:04}.tiff')
mshow(prb_abs*5000,True,vmax=7000,vmin=1000)
plt.savefig(f"figs/prbabs3.png", dpi=300, bbox_inches="tight", pad_inches=0.02)
mshow(prb_angle,True,vmax=2.4,vmin=-2.4)
plt.savefig(f"figs/prbangle3.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


In [ ]:
mpl.rcParams['font.size'] = 28  # optional
mpl.rcParams["xtick.labelsize"] = 28
mpl.rcParams["ytick.labelsize"] = 28
z1_ids = np.array([0,1,2,3])
str_z1_ids = ''.join(map(str, z1_ids + 1))
ndist = len(z1_ids)

rpos = np.load(f'/data2//vnikitin/brain_rec/20251115/Y350a1234/4500_2048_0_0.0_0.003_0.05_0.02_20_1.1_0/pos{iter:04}.npy')
with h5py.File(f'{path_out}/data{str_z1_ids}.h5') as fid:                
        pos = (fid[f'/exchange/cshifts_final'][:]).astype('float32')
rotation_center_shift = -8.780113000000028
rpos[:,:,1]-=rotation_center_shift
err = pos-rpos
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
l=ax.plot(err[::2,:,1], ".")
ax.set_xlabel('angle')
ax.set_ylabel('horizonal error')
labels = ["0", "1", "2", "3"]
leg = ax.legend(l, labels, title="distance",ncol=len(labels),loc='upper right',columnspacing=1,
    handletextpad=0.1,handlelength=0.6,fontsize=26,markerscale=4.0)
leg._legend_box.align = "left"
# ax[1].plot(err[::2,:,0], ".")
# ax[1].set_title("y")
# ax[0].grid()
ax.grid()
plt.savefig(f"figs/errx{k}.png", dpi=300, bbox_inches="tight", pad_inches=0.02)

fig, ax = plt.subplots(1, 1, figsize=(10, 4))
l=ax.plot(err[::2,:,0], ".")
ax.set_xlabel('angle')
ax.set_ylabel('vertical error')
labels = ["0", "1", "2", "3"]
leg = ax.legend(l, labels, title="distance",ncol=len(labels),loc='upper right',columnspacing=1,
    handletextpad=0.1,handlelength=0.6,fontsize=26,markerscale=4.0)
leg._legend_box.align = "left"
# plt.legend(markerscale=2.0)   # legend markers 2× larger
# ax[1].plot(err[::2,:,0], ".")
# ax[1].set_title("y")
# ax[0].grid()
ax.grid()
plt.savefig(f"figs/erry{k}.png", dpi=300, bbox_inches="tight", pad_inches=0.02)


In [ ]:
ss = 1024
bb=mrec[nz//2-ss:nz//2+ss,ny//2-ss:ny//2+ss,nx//2-ss:nx//2+ss]
aa=prec[nz//2-ss:nz//2+ss,ny//2-ss:ny//2+ss,nx//2-ss:nx//2+ss]

aa = (aa-mmin1)/(mmax1-mmin1)
bb = (bb-mmin2)/(mmax2-mmin2)
aa[aa<0] = 0
aa[aa>1] = 1
bb[bb<0] = 0
bb[bb>1] = 1
